
# Natural Language Processing

based on work by Magnus Pederson

## Imports

In [1]:
%matplotlib inline
import matplotlib.pyplot as plt
import tensorflow as tf
import numpy as np
import math

We need to import several things from Keras.

In [2]:
from tensorflow import keras
from tensorflow.keras.models import Sequential, save_model, load_model
from tensorflow.keras.layers import Dense, GRU, Embedding
from tensorflow.keras.optimizers import Adam, Ftrl, Adamax, SGD, Adadelta, Nadam, Optimizer, RMSprop, Adagrad
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint,CSVLogger, ReduceLROnPlateau,TensorBoard, LambdaCallback

from tensorflow.keras import backend as K
from tensorflow.keras.activations import relu, sigmoid ,softmax, tanh, hard_sigmoid, softsign, softplus, linear

from tensorflow.keras import backend as K
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import InputLayer, Input
from tensorflow.keras.layers import Reshape, MaxPooling2D
from tensorflow.keras.layers import Conv2D, Dense, Flatten
from tensorflow.keras.callbacks import TensorBoard
from tensorflow.keras.models import load_model

import skopt
from skopt import gp_minimize, forest_minimize
from skopt.space import Real, Categorical, Integer
from skopt.plots import plot_convergence
from skopt.plots import plot_objective, plot_evaluations
from skopt.plots import plot_histogram, plot_objective_2D
from skopt.utils import use_named_args
import sklearn.model_selection as sk
from sklearn.metrics import confusion_matrix, auc, roc_curve
import itertools
import matplotlib.pyplot as plt

This was developed using Python 3.6 (Anaconda) and package versions:

In [3]:
tf.__version__

'2.0.0'

In [4]:
tf.keras.__version__

'2.2.4-tf'

## Load Data

For Training use the twitter data set with sentiment analysis

In [5]:
import pandas as pd

file = '/Volumes/Elements/GitHub/twitter-project/Data Files/twitter_sentiment_learn.csv'

learning_df = pd.read_csv(file)
print(learning_df.shape)
print(learning_df.head())

(385042, 6)
   Unnamed: 0          TWID       NEG       NEU       POS  \
0           0  7.680000e+17  0.049398  0.861395  0.089207   
1           1  7.680000e+17  0.006598  0.046810  0.946591   
2           2  7.680000e+17  0.032333  0.850945  0.116722   
3           3  7.680000e+17  0.008090  0.042331  0.949579   
4           4  7.680000e+17  0.009325  0.940488  0.050187   

                                                text  
0  #Incredible #India #Atulya #Bharat - Land of S...  
1  RT @KendallHuntRPD: The #firstdayofschool for ...  
2  RT @abbiesf_: Kate wrights figure is all I wan...  
3  Josh Jenkins is looking forward to TAB Breeder...  
4  Robert Pattinson Gets Ready to Hop on a Plane ...  


Change this if you want the files saved in another directory.

## Load the training- and test-sets.

In [6]:
x = learning_df['text'].to_list()



learning_df[["bad", "meh", "good"]] = 0
for i in range(learning_df.shape[0]):
    if (learning_df.loc[i,"NEG"] >= learning_df.loc[i,"NEU"]) and (learning_df.loc[i,"NEG"] >= learning_df.loc[i,"POS"]):
        learning_df.loc[i,"bad"] = 1
    elif (learning_df.loc[i,"NEU"] >= learning_df.loc[i,"NEG"]) and (learning_df.loc[i,"NEU"] >= learning_df.loc[i,"POS"]):
        learning_df.loc[i,"meh"] = 1
    else:
        learning_df.loc[i,"good"] = 1

y = learning_df[["bad", "meh", "good"]]


##y = tarmac

x_train,x_test,y_train,y_test= sk.train_test_split(x,y,test_size=0.20, random_state = 42)

# Convert to numpy arrays.
y_train = np.array(y_train)
y_test = np.array(y_test)
##print(x[1])

print("Train-set size: ", len(x_train))
print("Test-set size:  ", len(x_test))

data_text = x_train + x_test

Train-set size:  308033
Test-set size:   77009


In [7]:
data_text = x_train + x_test

Print an example from the training-set to see that the data looks correct.

In [8]:
print("\n", x_train[1])
print("\n", learning_df.head())
print("\n", y_train[1:5])


 RT @khsaddampti: Indian army caputred hyderabad through operation polo #SunderLalCommissionReport https://t.co/c6XBmHyh46

    Unnamed: 0          TWID       NEG       NEU       POS  \
0           0  7.680000e+17  0.049398  0.861395  0.089207   
1           1  7.680000e+17  0.006598  0.046810  0.946591   
2           2  7.680000e+17  0.032333  0.850945  0.116722   
3           3  7.680000e+17  0.008090  0.042331  0.949579   
4           4  7.680000e+17  0.009325  0.940488  0.050187   

                                                text  bad  meh  good  
0  #Incredible #India #Atulya #Bharat - Land of S...    0    1     0  
1  RT @KendallHuntRPD: The #firstdayofschool for ...    0    0     1  
2  RT @abbiesf_: Kate wrights figure is all I wan...    0    1     0  
3  Josh Jenkins is looking forward to TAB Breeder...    0    0     1  
4  Robert Pattinson Gets Ready to Hop on a Plane ...    0    1     0  

 [[0 1 0]
 [0 1 0]
 [0 1 0]
 [0 1 0]]


The true "class" is a sentiment of the movie-review. It is a value of 0.0 for a negative sentiment and 1.0 for a positive sentiment. In this case the review is positive.

In [9]:
y_train[1]

array([0, 1, 0])

## Tokenizer

A neural network cannot work directly on text-strings so we must convert it somehow. There are two steps in this conversion, the first step is called the "tokenizer" which converts words to integers and is done on the data-set before it is input to the neural network. The second step is an integrated part of the neural network itself and is called the "embedding"-layer, which is described further below.

We may instruct the tokenizer to only use e.g. the 10000 most popular words from the data-set.

In [10]:
## can be played with 

num_words = 30000

tokenizer = Tokenizer(num_words=num_words,
                      filters='!"#$%&()*+,-./:;<=>?@[\\]^_`{|}~\t\n',
    lower=True, split=' ', char_level=False, oov_token=None)

##%%time
tokenizer.fit_on_texts(data_text)

x_train_tokens = tokenizer.texts_to_sequences(x_train)

x_test_tokens = tokenizer.texts_to_sequences(x_test)

The tokenizer can then be "fitted" to the data-set. This scans through all the text and strips it from unwanted characters such as punctuation, and also converts it to lower-case characters. The tokenizer then builds a vocabulary of all unique words along with various data-structures for accessing the data.

Note that we fit the tokenizer on the entire data-set so it gathers words from both the training- and test-data. This is OK as we are merely building a vocabulary and want it to be as complete as possible. The actual neural network will of course only be trained on the training-set.

If you want to use the entire vocabulary then set `num_words=None` above, and then it will automatically be set to the vocabulary-size here. (This is because of Keras' somewhat awkward implementation.)

In [11]:
print(num_words)
if num_words is None:
    num_words = len(tokenizer.word_index)
    print("the num_words is ", num_words)

30000


We can then inspect the vocabulary that has been gathered by the tokenizer. This is ordered by the number of occurrences of the words in the data-set. These integer-numbers are called word indices or "tokens" because they uniquely identify each word in the vocabulary.

We can then use the tokenizer to convert all texts in the training-set to lists of these tokens.

In [12]:
x_train_tokens = tokenizer.texts_to_sequences(x_train)

For example, here is a text from the training-set:

In [13]:
##print("\n",learning_df.loc[160625,:])
print("\n", x_train_tokens)
##print(y[1])

IOPub data rate exceeded.
The Jupyter server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--ServerApp.iopub_data_rate_limit`.

Current values:
ServerApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
ServerApp.rate_limit_window=3.0 (secs)



This text corresponds to the following list of tokens:

In [14]:
np.array(x_train_tokens[1])

array([   4, 1173, 1296, 7761,  295, 5603, 3413, 8189,    1,    2,    3])

We also need to convert the texts in the test-set to tokens.

In [15]:
x_test_tokens = tokenizer.texts_to_sequences(x_test)

## Padding and Truncating Data

The Recurrent Neural Network can take sequences of arbitrary length as input, but in order to use a whole batch of data, the sequences need to have the same length. There are two ways of achieving this: (A) Either we ensure that all sequences in the entire data-set have the same length, or (B) we write a custom data-generator that ensures the sequences have the same length within each batch.

Solution (A) is simpler but if we use the length of the longest sequence in the data-set, then we are wasting a lot of memory. This is particularly important for larger data-sets.

So in order to make a compromise, we will use a sequence-length that covers most sequences in the data-set, and we will then truncate longer sequences and pad shorter sequences.

First we count the number of tokens in all the sequences in the data-set.

In [16]:
num_tokens = [len(tokens) for tokens in x_train_tokens + x_test_tokens]
num_tokens = np.array(num_tokens)

The average number of tokens in a sequence is:

In [17]:
np.mean(num_tokens)

15.617023597425735

The maximum number of tokens in a sequence is:

In [18]:
np.max(num_tokens)

36

### The max number of tokens we will allow is set to the average plus 3 standard deviations.


# In this case, try the full amount

In [19]:
##max_tokens = np.mean(num_tokens) + 3 * np.std(num_tokens)
max_tokens = np.max(num_tokens)
max_tokens = math.floor(max_tokens)

This covers about 95% of the data-set.

In [20]:
np.sum(num_tokens < max_tokens) / len(num_tokens)

0.9999974028807247

When padding or truncating the sequences that have a different length, we need to determine if we want to do this padding or truncating 'pre' or 'post'. If a sequence is truncated, it means that a part of the sequence is simply thrown away. If a sequence is padded, it means that zeros are added to the sequence.

So the choice of 'pre' or 'post' can be important because it determines whether we throw away the first or last part of a sequence when truncating, and it determines whether we add zeros to the beginning or end of the sequence when padding. This may confuse the Recurrent Neural Network.

In [21]:
pad = 'pre'

In [22]:
x_train_pad = pad_sequences(x_train_tokens, maxlen=max_tokens,
                            padding=pad, truncating=pad)

In [23]:
x_test_pad = pad_sequences(x_test_tokens, maxlen=max_tokens,
                           padding=pad, truncating=pad)

We have now transformed the training-set into one big matrix of integers (tokens) with this shape:

In [24]:
x_train_pad.shape

(308033, 36)

The matrix for the test-set has the same shape:

In [25]:
x_test_pad.shape

(77009, 36)

For example, we had the following sequence of tokens above:

In [26]:
np.array(x_train_tokens[1])

array([   4, 1173, 1296, 7761,  295, 5603, 3413, 8189,    1,    2,    3])

This has simply been padded to create the following sequence. Note that when this is input to the Recurrent Neural Network, then it first inputs a lot of zeros. If we had padded 'post' then it would input the integer-tokens first and then a lot of zeros. This may confuse the Recurrent Neural Network.

In [27]:
x_train_pad[1]

array([   0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    4, 1173, 1296, 7761,  295, 5603, 3413, 8189,
          1,    2,    3], dtype=int32)

## Tokenizer Inverse Map

For some strange reason, the Keras implementation of a tokenizer does not seem to have the inverse mapping from integer-tokens back to words, which is needed to reconstruct text-strings from lists of tokens. So we make that mapping here.

In [28]:
idx = tokenizer.word_index
inverse_map = dict(zip(idx.values(), idx.keys()))

Helper-function for converting a list of tokens back to a string of words.

In [29]:
def tokens_to_string(tokens):
    # Map from tokens back to words.
    words = [inverse_map[token] for token in tokens if token != 0]
    
    # Concatenate all words.
    text = " ".join(words)

    return text

For example, this is the original text from the data-set:

In [30]:
x_train[1]

'RT @khsaddampti: Indian army caputred hyderabad through operation polo #SunderLalCommissionReport https://t.co/c6XBmHyh46'

We can recreate this text except for punctuation and other symbols, by converting the list of tokens back to words:

In [31]:
tokens_to_string(x_train_tokens[1])

'rt indian army hyderabad through operation polo sunderlalcommissionreport https t co'

## Create the Recurrent Neural Network

We are now ready to create the Recurrent Neural Network (RNN). We will use the Keras API for this because of its simplicity. See Tutorial #03-C for a tutorial on Keras.

In [32]:
model = Sequential()

The first layer in the RNN is a so-called Embedding-layer which converts each integer-token into a vector of values. This is necessary because the integer-tokens may take on values between 0 and 10000 for a vocabulary of 10000 words. The RNN cannot work on values in such a wide range. The embedding-layer is trained as a part of the RNN and will learn to map words with similar semantic meanings to similar embedding-vectors, as will be shown further below.

First we define the size of the embedding-vector for each integer-token. In this case we have set it to 8, so that each integer-token will be converted to a vector of length 8. The values of the embedding-vector will generally fall roughly between -1.0 and 1.0, although they may exceed these values somewhat.

The size of the embedding-vector is typically selected between 100-300, but it seems to work reasonably well with small values for Sentiment Analysis.

In [33]:
figure_of_merit = 2 * max_tokens ## was 100
first_layer = math.floor(max_tokens/2) + 2
second_layer = math.floor(max_tokens/3) + 2
third_layer = math.floor(max_tokens/4) + 3
fourth_layer = math.floor(max_tokens/5) + 3
embedding_size = figure_of_merit

The embedding-layer also needs to know the number of words in the vocabulary (`num_words`) and the length of the padded token-sequences (`max_tokens`). We also give this layer a name because we need to retrieve its weights further below.

In [34]:
model.add(Embedding(input_dim=num_words,## was num_words
                    output_dim=embedding_size,                    
                    input_length=max_tokens,
                   name='layer_embedding'))
##model.add(Flatten())

##model = tf.keras.Sequential([
##    tf.keras.layers.Embedding(input_dim = num_words, output_dim = embedding_size, input_length=max_tokens),
##    tf.keras.layers.Flatten(),
##    tf.keras.layers.Dense(30, activation='relu'),
##    tf.keras.layers.Dense(60, activation='sigmoid')
##    tf.keras.layers.Dense(15, activation='relu')
##    tf.keras.layers.Dense(3, activation='sigmoid')
##])

2022-05-18 12:10:51.837734: I tensorflow/core/platform/cpu_feature_guard.cc:145] This TensorFlow binary is optimized with Intel(R) MKL-DNN to use the following CPU instructions in performance critical operations:  SSE4.1 SSE4.2 AVX AVX2 FMA
To enable them in non-MKL-DNN operations, rebuild TensorFlow with the appropriate compiler flags.
2022-05-18 12:10:51.838098: I tensorflow/core/common_runtime/process_util.cc:115] Creating new thread pool with default inter op setting: 16. Tune using inter_op_parallelism_threads for best performance.


We can now add the first Gated Recurrent Unit (GRU) to the network. This will have 16 outputs. Because we will add a second GRU after this one, we need to return sequences of data because the next GRU expects sequences as its input.

In [35]:
model.add(GRU(units=first_layer, 
              activation = 'tanh', ## was tanh
              recurrent_activation = 'softmax', 
              return_sequences=True))
##model.add(Dense(first_layer, activation='softmax'))

This adds the second GRU with 8 output units. This will be followed by another GRU so it must also return sequences.

In [36]:
model.add(GRU(units=second_layer, activation = 'tanh' , ## was tanh
              recurrent_activation = 'softmax', 
              return_sequences=True))
##model.add(Dense(second_layer, activation='sigmoid'))

This adds the third and final GRU with 4 output units. This will be followed by a dense-layer, so it should only give the final output of the GRU and not a whole sequence of outputs.

In [37]:
model.add(GRU(units=third_layer, activation = 'tanh', ## was tanh
              recurrent_activation = 'softmax', 
              return_sequences = True))
model.add(GRU(units=fourth_layer, activation = 'tanh', ## was tanh
              recurrent_activation = 'sigmoid', return_sequences = False))

## Add a fully-connected / dense layer which computes a value between 0 and 1.0 that will be used as the classification output.

In [38]:
model.add(Dense(3, activation='softmax')) ## was 3

Use the Adam optimizer with the given learning-rate.

In [39]:
learning_rat = 1e-3
optimizer = Nadam(learning_rate=learning_rat)

Compile the Keras model so it is ready for training.

In [40]:
model.compile(loss='categorical_crossentropy',
              optimizer=optimizer,
             metrics=['accuracy'])
##model.compile(loss='binary_crossentropy',
##              optimizer=optimizer,
##              metrics=['accuracy'])

In [41]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
layer_embedding (Embedding)  (None, 36, 72)            2160000   
_________________________________________________________________
gru (GRU)                    (None, 36, 20)            5640      
_________________________________________________________________
gru_1 (GRU)                  (None, 36, 14)            1512      
_________________________________________________________________
gru_2 (GRU)                  (None, 36, 12)            1008      
_________________________________________________________________
gru_3 (GRU)                  (None, 10)                720       
_________________________________________________________________
dense (Dense)                (None, 3)                 33        
Total params: 2,168,913
Trainable params: 2,168,913
Non-trainable params: 0
______________________________________________

## make callbacks

In [42]:
callbackx = tf.keras.callbacks.EarlyStopping(monitor='val_accuracy', 
                                             patience =1,
                                            restore_best_weights = True)
        
class myCallback(tf.keras.callbacks.Callback):
  def on_epoch_end(self, epoch, logs={}):
    if(logs.get('val_accuracy') > 0.95):
      print("\nReached 95% val_accuracy, so slowing the learning rate and keeping Nadam optimizer.")
      optimizer = Nadam(learning_rate = 0.4*learning_rat)
      self.model.stop_training = False
    if(logs.get('val_accuracy') > 0.970):
      print("\nReached 987 val_accuracy, so slowing the learning rate and keeping Nadam optimizer.")
      optimizer = Nadam(learning_rate = 0.1*learning_rat)
      self.model.stop_training = False
call_it = myCallback()

## Train the Recurrent Neural Network

We can now train the model. Note that we are using the data-set with the padded sequences. We use 5% of the training-set as a small validation-set, so we have a rough idea whether the model is generalizing well or if it is perhaps over-fitting to the training-set.

%%time
model.fit(x_train_pad, y_train,
          validation_split=0.15, epochs=10, batch_size= 256,
            callbacks = [callbackx, call_it])

## Performance on Test-Set

Now that the model has been trained we can calculate its classification accuracy on the test-set.

In [43]:
%%time
model.fit(x_train_pad, y_train,
          validation_split=0.25, epochs=40, batch_size=100, verbose = 2,
            callbacks = [call_it, callbackx])

Train on 231024 samples, validate on 77009 samples
Epoch 1/40


2022-05-18 12:10:59.407053: W tensorflow/core/grappler/optimizers/implementation_selector.cc:310] Skipping optimization due to error while loading function libraries: Invalid argument: Functions '__inference___backward_cudnn_gru_with_fallback_6612_6751' and '__inference___backward_standard_gru_6867_7426_specialized_for_StatefulPartitionedCall_at___inference_distributed_function_9726' both implement 'gru_5ccf5926-3e21-40f9-9294-b26115528ca6' but their signatures do not match.
2022-05-18 12:26:37.941419: W tensorflow/core/grappler/optimizers/implementation_selector.cc:310] Skipping optimization due to error while loading function libraries: Invalid argument: Functions '__inference_cudnn_gru_with_fallback_17545' and '__inference_standard_gru_17456_specialized_for_sequential_gru_3_StatefulPartitionedCall_at___inference_distributed_function_17769' both implement 'gru_51df576a-9a1a-4dc6-8fe4-6438e712cb7b' but their signatures do not match.



Reached 95% val_accuracy, so slowing the learning rate and keeping Nadam optimizer.
231024/231024 - 1049s - loss: 0.2240 - accuracy: 0.9246 - val_loss: 0.1374 - val_accuracy: 0.9587
Epoch 2/40

Reached 95% val_accuracy, so slowing the learning rate and keeping Nadam optimizer.
231024/231024 - 865s - loss: 0.1081 - accuracy: 0.9685 - val_loss: 0.1291 - val_accuracy: 0.9627
Epoch 3/40

Reached 95% val_accuracy, so slowing the learning rate and keeping Nadam optimizer.
231024/231024 - 937s - loss: 0.0867 - accuracy: 0.9759 - val_loss: 0.1300 - val_accuracy: 0.9622
CPU times: user 2h 28min 23s, sys: 6h 51min 56s, total: 9h 20min 19s
Wall time: 47min 30s


In [44]:

result = model.evaluate(x_test_pad, y_test)
print("and now")

77009/1 [===============================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================

In [45]:
print("Accuracy: {0:.2%}".format(result[1]))





Accuracy: 96.45%


## save model

In [46]:
##from keras.models import models_from_json
model_json = model.to_json()
with open ("model_cat.json", "w") as json_file:
    json_file.write(model_json)
model.save_weights("model_cat.h5")

model.save("model_cat.h5")
print(model_json)
print("saved model")

{"class_name": "Sequential", "config": {"name": "sequential", "layers": [{"class_name": "Embedding", "config": {"name": "layer_embedding", "trainable": true, "batch_input_shape": [null, 36], "dtype": "float32", "input_dim": 30000, "output_dim": 72, "embeddings_initializer": {"class_name": "RandomUniform", "config": {"minval": -0.05, "maxval": 0.05, "seed": null}}, "embeddings_regularizer": null, "activity_regularizer": null, "embeddings_constraint": null, "mask_zero": false, "input_length": 36}}, {"class_name": "GRU", "config": {"name": "gru", "trainable": true, "dtype": "float32", "return_sequences": true, "return_state": false, "go_backwards": false, "stateful": false, "unroll": false, "time_major": false, "units": 20, "activation": "tanh", "recurrent_activation": "softmax", "use_bias": true, "kernel_initializer": {"class_name": "GlorotUniform", "config": {"seed": null}}, "recurrent_initializer": {"class_name": "Orthogonal", "config": {"gain": 1.0, "seed": null}}, "bias_initializer":

## Example of Mis-Classified Text

In order to show an example of mis-classified text, we first calculate the predicted sentiment for the first 1000 texts in the test-set.

In [47]:
## Load the saved model and see if the results differ

model_file = 'model_cat.h5'

loaded_model = load_model(
    model_file,
    custom_objects = None,
    compile = True
)



AttributeError: 'str' object has no attribute 'decode'

In [ ]:
%%time
##y_pred = model.predict(x=x_test_pad[0:1000])
##y_pred = loaded_model.predict(x=x_test_pad[0:1000])
##y_pred = y_pred.T[0]


These predicted numbers fall between 0.0 and 1.0. We use a cutoff / threshold and say that all values above 0.5 are taken to be 1.0 and all values below 0.5 are taken to be 0.0. This gives us a predicted "class" of either 0.0 or 1.0.

## Load the model and use that to create a confusion matrix

In [ ]:
preddy = loaded_model.predict(x_test_pad)

preddy = np.array(preddy)

catzy_df = pd.DataFrame(preddy, columns = ['NEG', 'NEU', 'POS'])

catzy_df[["neg", "neu", "pos"]] = 0
for i in range(catzy_df.shape[0]):
    if (catzy_df.loc[i,"NEG"] >= catzy_df.loc[i,"NEU"]) and (catzy_df.loc[i,"NEG"] >= catzy_df.loc[i,"POS"]):
        catzy_df.loc[i,"neg"] = 1
    elif (catzy_df.loc[i,"NEU"] >= catzy_df.loc[i,"NEG"]) and (catzy_df.loc[i,"NEU"] >= catzy_df.loc[i,"POS"]):
        catzy_df.loc[i,"neu"] = 1
    else:
        catzy_df.loc[i,"pos"] = 1

preddy = catzy_df[["neg", "neu", "pos"]]



print("The head of preddy is \n",preddy.head())
##print("The head of y_test is \n",y_test.head())

In [ ]:
from sklearn.metrics import classification_report, precision_score, recall_score, f1_score, precision_recall_fscore_support, multilabel_confusion_matrix

print('Precision                                   : %.3f'%precision_score(y_test, preddy, average = 'weighted'))
print('Recall                                      : %.3f'%recall_score(y_test, preddy, average = 'weighted'))
print('F1-Score                                    : %.3f'%f1_score(y_test, preddy, average = 'weighted'))
print('\nPrecision Recall F1-Score Support Per Class : \n',precision_recall_fscore_support(y_test, preddy, average = 'weighted'))
print('\nClassification Report                       : ')
print(classification_report(y_test, preddy))


turkey = 0*preddy['neg'] + preddy['neu'] + 2*preddy['pos']

print("the value counts for turkey are \n", turkey.value_counts())

grubby = np.array(y_test)

y_catty = pd.DataFrame(grubby, columns = ['neg', 'neu', 'pos'])

doggy = 0*y_catty['neg'] + y_catty['neu'] + 2*y_catty['pos']

print("the value counts for doggy are \n", doggy.value_counts())

##cm = multilabel_confusion_matrix(y_true=y_test, y_pred=preddy)

import seaborn as sns

cm = confusion_matrix(doggy, turkey)
cm_df = pd.DataFrame(cm,
                     index = ['Negative','Neutral','Positive'], 
                     columns = ['Negative','Neutral','Positive'])

plt.figure(figsize=(12,9))
sns.heatmap(cm_df, annot=True)
plt.title('Confusion Matrix')
plt.ylabel('Actual Values')
plt.xlabel('Predicted Values')
plt.show()


## New Data

Let us try and classify new texts that we make up. Some of these are obvious, while others use negation and sarcasm to try and confuse the model into mis-classifying the text.

## Let's do Amazon with the category stuff

In [ ]:
amzn_file = '/Volumes/Elements/GitHub/twitter-project/Data Files/Amazon_df_json.csv'

amzn_df =  pd.read_csv(amzn_file, header=0, index_col=0, parse_dates=True)

am_columns = ['numbers', 'created_at','full_text', 'retweet_count', 'user_id', 
           'user.favourites_count', 'user.followers_count', 'AMZN', 'Amazon', 'dupe']
amzn_df.columns = am_columns

amzn_text = amzn_df['full_text'].astype(str)


amzn_x = amzn_text.to_list()

bob = amzn_x[0:7]

print(type(bob))

num_words = 30000

tokenizer = Tokenizer(num_words=num_words)

##%%time
tokenizer.fit_on_texts(amzn_x)

amzn_train_tokens = tokenizer.texts_to_sequences(amzn_x)

amzn_pad = pad_sequences(amzn_train_tokens, maxlen=max_tokens,
                           padding=pad, truncating=pad)

amzn_sent = loaded_model.predict(x=amzn_pad)

print(amzn_sent[1:3])

predictions = pd.DataFrame(amzn_sent, columns  = ['neg', 'neu', 'pos'])

amzn_df_nn = amzn_df.join(predictions)
amzn_df_nn.columns = ['number','created_at','full_text', 'retweet_count', 'user_id', 'user.favourites_count', 'user.followers_count','Symbol','Name', 'dupe', 'neg', 'neu', 'pos']

##    print("\nThe head of the ", i, " dataframe is \n", Krusty[i].head())

amzn_df_nn.to_csv('/Volumes/Elements/GitHub/twitter-project/Data Files/Amazon_nn_scored_cat.csv', header = True)

print('the shape of Amazon_df_nn is ', amzn_df_nn.shape)

## KMI 

In [ ]:
KMI_file = '/Volumes/Elements/GitHub/twitter-project/Data Files/KMI_df_json.csv'

KMI_df =  pd.read_csv(KMI_file, header=0, index_col=0, parse_dates=True)

KMI_columns = ['numbers', 'created_at','full_text', 'retweet_count', 'user_id', 
           'user.favourites_count', 'user.followers_count', 'KMI', 'Kinder_Morgan', 'dupe']
KMI_df.columns = KMI_columns

KMI_text = KMI_df['full_text'].astype(str)


KMI_x = KMI_text.to_list()

print(type(KMI_x))

num_words = 30000

tokenizer = Tokenizer(num_words=num_words)

##%%time
tokenizer.fit_on_texts(KMI_x)

KMI_train_tokens = tokenizer.texts_to_sequences(KMI_x)

KMI_pad = pad_sequences(KMI_train_tokens, maxlen=max_tokens,
                           padding=pad, truncating=pad)

KMI_sent = loaded_model.predict(x=KMI_pad)

print(KMI_sent[1:3])

predictions = pd.DataFrame(KMI_sent, columns  = ['neg', 'neu', 'pos'])

KMI_df_nn = KMI_df.join(predictions)
KMI_df_nn.columns = ['number','created_at','full_text', 'retweet_count', 'user_id', 'user.favourites_count', 'user.followers_count','Symbol', 'Name','dupe', 'neg', 'neu', 'pos']

##    print("\nThe head of the ", i, " dataframe is \n", Krusty[i].head())

KMI_df_nn.to_csv('/Volumes/Elements/GitHub/twitter-project/Data Files/KMI_nn_scored_cat.csv', header = True)

print('the shape of KMI _df_nn is ', KMI_df_nn.shape)

## CrowdSource

In [ ]:
CRWD_file = '/Volumes/Elements/GitHub/twitter-project/Data Files/CRWD_df_json.csv'

CRWD_df =  pd.read_csv(CRWD_file, header=0, index_col=0, parse_dates=True)

CRWD_columns = ['numbers', 'created_at','full_text', 'retweet_count', 'user_id', 
           'user.favourites_count', 'user.followers_count', 'CRWD', 'CrowdSource', 'dupe']
CRWD_df.columns = CRWD_columns

CRWD_text = CRWD_df['full_text'].astype(str)


CRWD_x = CRWD_text.to_list()

print(type(CRWD_x))

num_words = 30000

tokenizer = Tokenizer(num_words=num_words)

##%%time
tokenizer.fit_on_texts(CRWD_x)

CRWD_train_tokens = tokenizer.texts_to_sequences(CRWD_x)

CRWD_pad = pad_sequences(CRWD_train_tokens, maxlen=max_tokens,
                           padding=pad, truncating=pad)

CRWD_sent = loaded_model.predict(x=CRWD_pad)

print(CRWD_sent[1:3])

predictions = pd.DataFrame(CRWD_sent, columns  = ['neg', 'neu', 'pos'])

CRWD_df_nn = CRWD_df.join(predictions)
CRWD_df_nn.columns = ['number','created_at','full_text', 'retweet_count', 'user_id', 'user.favourites_count', 'user.followers_count','Symbol', 'Name','dupe', 'neg', 'neu', 'pos']

##    print("\nThe head of the ", i, " dataframe is \n", Krusty[i].head())

CRWD_df_nn.to_csv('/Volumes/Elements/GitHub/twitter-project/Data Files/CRWD_nn_scored_cat.csv', header = True)

print('the shape of CRWD_df_nn is ', CRWD_df_nn.shape)

## Now Appian¶

In [ ]:
APPN_file = '/Volumes/Elements/GitHub/twitter-project/Data Files/APPN_df_json.csv'

APPN_df =  pd.read_csv(APPN_file, header=0, index_col=0, parse_dates=True)

APPN_columns = ['numbers', 'created_at','full_text', 'retweet_count', 'user_id', 
           'user.favourites_count', 'user.followers_count', 'APPN', 'Appian', 'dupe']
APPN_df.columns =APPN_columns

APPN_text = APPN_df['full_text'].astype(str)


APPN_x = APPN_text.to_list()

print(type(APPN_x))

num_words = 30000

tokenizer = Tokenizer(num_words=num_words)

##%%time
tokenizer.fit_on_texts(APPN_x)

APPN_train_tokens = tokenizer.texts_to_sequences(APPN_x)

APPN_pad = pad_sequences(APPN_train_tokens, maxlen=max_tokens,
                           padding=pad, truncating=pad)

APPN_sent = loaded_model.predict(x=APPN_pad)

print(APPN_sent[1:3])

predictions = pd.DataFrame(APPN_sent, columns  = ['neg', 'neu', 'pos'])

APPN_df_nn = APPN_df.join(predictions)
APPN_df_nn.columns = ['number','created_at','full_text', 'retweet_count', 'user_id', 'user.favourites_count', 'user.followers_count','Symbol', 'Name','dupe', 'neg', 'neu', 'pos']

##    print("\nThe head of the ", i, " dataframe is \n", Krusty[i].head())

APPN_df_nn.to_csv('/Volumes/Elements/GitHub/twitter-project/Data Files/APPN_nn_scored_cat.csv', header = True)

print('the shape of APPN_df_nn is ', APPN_df_nn.shape)



## Inspire

In [ ]:
INSP_file = '/Volumes/Elements/GitHub/twitter-project/Data Files/INSP_df_json.csv'

INSP_df =  pd.read_csv(INSP_file, header=0, index_col=0, parse_dates=True)

INSP_columns = ['numbers', 'created_at','full_text', 'retweet_count', 'user_id', 
           'user.favourites_count', 'user.followers_count', 'INSP', 'Inspire_Systems', 'dupe']
INSP_df.columns = INSP_columns

INSP_text = INSP_df['full_text'].astype(str)


INSP_x = INSP_text.to_list()

print(type(INSP_x))

num_words = 30000

tokenizer = Tokenizer(num_words=num_words)

##%%time
tokenizer.fit_on_texts(INSP_x)

INSP_train_tokens = tokenizer.texts_to_sequences(INSP_x)

INSP_pad = pad_sequences(INSP_train_tokens, maxlen=max_tokens,
                           padding=pad, truncating=pad)

INSP_sent = loaded_model.predict(x=INSP_pad)

print(INSP_sent[1:3])

predictions = pd.DataFrame(INSP_sent, columns  = ['neg', 'neu', 'pos'])

INSP_df_nn = INSP_df.join(predictions)
INSP_df_nn.columns = ['number','created_at','full_text', 'retweet_count', 'user_id', 'user.favourites_count', 'user.followers_count','Symbol', 'Name','dupe', 'neg', 'neu', 'pos']

##    print("\nThe head of the ", i, " dataframe is \n", Krusty[i].head())

INSP_df_nn.to_csv('/Volumes/Elements/GitHub/twitter-project/Data Files/INSP_nn_scored_cat.csv', header = True)

print('the shape of INSP_df_nn is ', INSP_df_nn.shape)



## Sanofy

In [ ]:
SNY_file = '/Volumes/Elements/GitHub/twitter-project/Data Files/SNY_df_json.csv'

SNY_df =  pd.read_csv(SNY_file, header=0, index_col=0, parse_dates=True)

SNY_columns = ['numbers', 'created_at','full_text', 'retweet_count', 'user_id', 
           'user.favourites_count', 'user.followers_count', 'SNY', 'Sanofy', 'dupe']
SNY_df.columns = SNY_columns

SNY_text = SNY_df['full_text'].astype(str)


SNY_x = SNY_text.to_list()

print(type(SNY_x))

num_words = 30000

tokenizer = Tokenizer(num_words=num_words)

##%%time
tokenizer.fit_on_texts(SNY_x)

SNY_train_tokens = tokenizer.texts_to_sequences(SNY_x)

SNY_pad = pad_sequences(SNY_train_tokens, maxlen=max_tokens,
                           padding=pad, truncating=pad)

SNY_sent = loaded_model.predict(x=SNY_pad)

print(SNY_sent[1:3])

predictions = pd.DataFrame(SNY_sent, columns  = ['neg', 'neu', 'pos'])

SNY_df_nn = SNY_df.join(predictions)
SNY_df_nn.columns = ['number','created_at','full_text', 'retweet_count', 'user_id', 'user.favourites_count', 'user.followers_count','Symbol', 'Name','dupe', 'neg', 'neu', 'pos']

##    print("\nThe head of the ", i, " dataframe is \n", Krusty[i].head())

SNY_df_nn.to_csv('/Volumes/Elements/GitHub/twitter-project/Data Files/SNY_nn_scored_cat.csv', header = True)

print('the shape of SNY_df_nn is ', SNY_df_nn.shape)



## Everbridge

In [ ]:
EVBG_file = '/Volumes/Elements/GitHub/twitter-project/Data Files/EVBG_df_json.csv'

EVBG_df =  pd.read_csv(EVBG_file, header=0, index_col=0, parse_dates=True)

EVBG_columns = ['numbers', 'created_at','full_text', 'retweet_count', 'user_id', 
           'user.favourites_count', 'user.followers_count', 'EVBG', 'Everbridge', 'dupe']
EVBG_df.columns = EVBG_columns

EVBG_text = EVBG_df['full_text'].astype(str)


EVBG_x = EVBG_text.to_list()

print(type(EVBG_x))

num_words = 30000

tokenizer = Tokenizer(num_words=num_words)

##%%time
tokenizer.fit_on_texts(EVBG_x)

EVBG_train_tokens = tokenizer.texts_to_sequences(EVBG_x)

EVBG_pad = pad_sequences(EVBG_train_tokens, maxlen=max_tokens,
                           padding=pad, truncating=pad)

EVBG_sent = loaded_model.predict(x=EVBG_pad)

print(EVBG_sent[1:3])

predictions = pd.DataFrame(EVBG_sent, columns  = ['neg', 'neu', 'pos'])

EVBG_df_nn = EVBG_df.join(predictions)
EVBG_df_nn.columns = ['number','created_at','full_text', 'retweet_count', 'user_id', 'user.favourites_count', 'user.followers_count','Symbol', 'Name','dupe', 'neg', 'neu', 'pos']

##    print("\nThe head of the ", i, " dataframe is \n", Krusty[i].head())

EVBG_df_nn.to_csv('/Volumes/Elements/GitHub/twitter-project/Data Files/EVBG_nn_scored_cat.csv', header = True)

print('the shape of EVBG_df_nn is ', EVBG_df_nn.shape)



## Exxon

In [ ]:
XOM_file = '/Volumes/Elements/GitHub/twitter-project/Data Files/XOM_df_json.csv'

XOM_df =  pd.read_csv(XOM_file, header=0, index_col=0, parse_dates=True)

XOM_columns = ['numbers', 'created_at','full_text', 'retweet_count', 'user_id', 
           'user.favourites_count', 'user.followers_count', 'XOM', 'Exxon', 'dupe']
XOM_df.columns = XOM_columns

XOM_text = XOM_df['full_text'].astype(str)


XOM_x = XOM_text.to_list()

print(type(XOM_x))

num_words = 30000

tokenizer = Tokenizer(num_words=num_words)

##%%time
tokenizer.fit_on_texts(XOM_x)

XOM_train_tokens = tokenizer.texts_to_sequences(XOM_x)

XOM_pad = pad_sequences(XOM_train_tokens, maxlen=max_tokens,
                           padding=pad, truncating=pad)

XOM_sent = loaded_model.predict(x=XOM_pad)

print(XOM_sent[1:3])

predictions = pd.DataFrame(XOM_sent, columns  = ['neg', 'neu', 'pos'])

XOM_df_nn = XOM_df.join(predictions)
XOM_df_nn.columns = ['number','created_at','full_text', 'retweet_count', 'user_id', 'user.favourites_count', 'user.followers_count','Symbol', 'Name','dupe', 'neg', 'neu', 'pos']

##    print("\nThe head of the ", i, " dataframe is \n", Krusty[i].head())

XOM_df_nn.to_csv('/Volumes/Elements/GitHub/twitter-project/Data Files/XOM_nn_scored_cat.csv', header = True)

print('the shape of XOM_df_nn is ', XOM_df_nn.shape)



## And Finally, GMED

In [ ]:
GMED_file = '/Volumes/Elements/GitHub/twitter-project/Data Files/GMED_df_json.csv'

GMED_df =  pd.read_csv(GMED_file, header=0, index_col=0, parse_dates=True)

GMED_columns = ['numbers', 'created_at','full_text', 'retweet_count', 'user_id', 
           'user.favourites_count', 'user.followers_count', 'GMED' , 'Global', 'dupe']
GMED_df.columns = GMED_columns

GMED_text = GMED_df['full_text'].astype(str)


GMED_x = GMED_text.to_list()

print(type(GMED_x))

num_words = 30000

tokenizer = Tokenizer(num_words=num_words)

##%%time
tokenizer.fit_on_texts(GMED_x)

GMED_train_tokens = tokenizer.texts_to_sequences(GMED_x)

GMED_pad = pad_sequences(GMED_train_tokens, maxlen=max_tokens,
                           padding=pad, truncating=pad)

GMED_sent = loaded_model.predict(x=GMED_pad)

print(GMED_sent[1:3])

predictions = pd.DataFrame(GMED_sent, columns  = ['neg', 'neu', 'pos'])

GMED_df_nn = GMED_df.join(predictions)
GMED_df_nn.columns = ['number','created_at','full_text', 'retweet_count', 'user_id', 'user.favourites_count', 'user.followers_count','Symbol', 'Name','dupe', 'neg', 'neu', 'pos']

##    print("\nThe head of the ", i, " dataframe is \n", Krusty[i].head())

GMED_df_nn.to_csv('/Volumes/Elements/GitHub/twitter-project/Data Files/GMED_nn_scored_cat.csv', header = True)

print('the shape of GMED_df_nn is ', GMED_df_nn.shape)



In [ ]:
text1 = "This movie is fantastic! I really like it because it is so good!"
text2 = "Good movie!"
text3 = "Maybe I like this movie."
text4 = "Meh ..."
text5 = "If I were a drunk teenager then this movie might be good."
text6 = "Bad movie!"
text7 = "Not a good movie!"
text8 = "This movie really sucks! Can I get my money back please?"
text9 = 'This was an awesome movie. I watch it twice my time watching this beautiful movie if I have known it was this good'
text10 = 'One of the worst movies of all time. I cannot believe I wasted two hours of my life for this movie'
text11 = 'At no point can I say I loved this movie.'
text12 = 'No complaint about this movie could be justified, just pure perfection.'

texts = [text1, text2, text3, text4, text5, text6, text7, text8, text9, text10, text11, text12]

We first convert these texts to arrays of integer-tokens because that is needed by the model.

In [ ]:
tokens = tokenizer.texts_to_sequences(texts)

To input texts with different lengths into the model, we also need to pad and truncate them.

In [ ]:
tokens_pad = pad_sequences(tokens, maxlen=max_tokens,
                           padding=pad, truncating=pad)
tokens_pad.shape

We can now use the trained model to predict the sentiment for these texts.

In [ ]:
## Note, using the loaded prediciton model.

loaded_model.predict(tokens_pad)

A value close to 0.0 means a negative sentiment and a value close to 1.0 means a positive sentiment. These numbers will vary every time you train the model.

## Embeddings

The model cannot work on integer-tokens directly, because they are integer values that may range between 0 and the number of words in our vocabulary, e.g. 10000. So we need to convert the integer-tokens into vectors of values that are roughly between -1.0 and 1.0 which can be used as input to a neural network.

This mapping from integer-tokens to real-valued vectors is also called an "embedding". It is essentially just a matrix where each row contains the vector-mapping of a single token. This means we can quickly lookup the mapping of each integer-token by simply using the token as an index into the matrix. The embeddings are learned along with the rest of the model during training.

Ideally the embedding would learn a mapping where words that are similar in meaning also have similar embedding-values. Let us investigate if that has happened here.

First we need to get the embedding-layer from the model:

In [ ]:
layer_embedding = model.get_layer('layer_embedding')

We can then get the weights used for the mapping done by the embedding-layer.

In [ ]:
weights_embedding = layer_embedding.get_weights()[0]

Note that the weights are actually just a matrix with the number of words in the vocabulary times the vector length for each embedding. That's because it is basically just a lookup-matrix.

In [ ]:
weights_embedding.shape

Let us get the integer-token for the word 'good', which is just an index into the vocabulary.

In [ ]:
token_good = tokenizer.word_index['good']
token_good

Let us also get the integer-token for the word 'great'.

In [ ]:
token_great = tokenizer.word_index['great']
token_great

These integertokens may be far apart and will depend on the frequency of those words in the data-set.

Now let us compare the vector-embeddings for the words 'good' and 'great'. Several of these values are similar, although some values are quite different. Note that these values will change every time you train the model.

In [ ]:
weights_embedding[token_good]

In [ ]:
weights_embedding[token_great]

Similarly, we can compare the embeddings for the words 'bad' and 'horrible'.

In [ ]:
##token_bad = tokenizer.word_index['bad']
##token_horrible = tokenizer.word_index['horrible']

In [ ]:
##weights_embedding[token_bad]

In [ ]:
##weights_embedding[token_horrible]

### Sorted Words

We can also sort all the words in the vocabulary according to their "similarity" in the embedding-space. We want to see if words that have similar embedding-vectors also have similar meanings.

Similarity of embedding-vectors can be measured by different metrics, e.g. Euclidean distance or cosine distance.

We have a helper-function for calculating these distances and printing the words in sorted order.

In [ ]:
def print_sorted_words(word, metric='cosine'):
    """
    Print the words in the vocabulary sorted according to their
    embedding-distance to the given word.
    Different metrics can be used, e.g. 'cosine' or 'euclidean'.
    """

    # Get the token (i.e. integer ID) for the given word.
    token = tokenizer.word_index[word]

    # Get the embedding for the given word. Note that the
    # embedding-weight-matrix is indexed by the word-tokens
    # which are integer IDs.
    embedding = weights_embedding[token]

    # Calculate the distance between the embeddings for
    # this word and all other words in the vocabulary.
    distances = cdist(weights_embedding, [embedding],
                      metric=metric).T[0]
    
    # Get an index sorted according to the embedding-distances.
    # These are the tokens (integer IDs) for words in the vocabulary.
    sorted_index = np.argsort(distances)
    
    # Sort the embedding-distances.
    sorted_distances = distances[sorted_index]
    
    # Sort all the words in the vocabulary according to their
    # embedding-distance. This is a bit excessive because we
    # will only print the top and bottom words.
    sorted_words = [inverse_map[token] for token in sorted_index
                    if token != 0]

    # Helper-function for printing words and embedding-distances.
    def _print_words(words, distances):
        for word, distance in zip(words, distances):
            print("{0:.3f} - {1}".format(distance, word))

    # Number of words to print from the top and bottom of the list.
    k = 10

    print("Distance from '{0}':".format(word))

    # Print the words with smallest embedding-distance.
    _print_words(sorted_words[0:k], sorted_distances[0:k])

    print("...")

    # Print the words with highest embedding-distance.
    _print_words(sorted_words[-k:], sorted_distances[-k:])

We can then print the words that are near and far from the word 'great' in terms of their vector-embeddings. Note that these may change each time you train the model.

In [ ]:
##print_sorted_words('great', metric='cosine')

Similarly, we can print the words that are near and far from the word 'worst' in terms of their vector-embeddings.

In [ ]:
##print_sorted_words('worst', metric='cosine')

In [ ]:
import datetime
now = datetime.datetime.now()
print("\n This finished at ", now)


## Conclusion

This tutorial showed the basic methods for doing Natural Language Processing (NLP) using a Recurrent Neural Network with integer-tokens and an embedding layer. This was used to do sentiment analysis of movie reviews from IMDB. It works reasonably well if the hyper-parameters are chosen properly. But it is important to understand that this is not human-like comprehension of text. The system does not have any real understanding of the text. It is just a clever way of doing pattern-recognition.

## Exercises

These are a few suggestions for exercises that may help improve your skills with TensorFlow. It is important to get hands-on experience with TensorFlow in order to learn how to use it properly.

You may want to backup this Notebook before making any changes.

* Run more training-epochs. Does it improve performance?
* If your model overfits the training-data, try using dropout-layers and dropout inside the GRU.
* Increase or decrease the number of words in the vocabulary. This is done when the `Tokenizer` is initialized. Does it affect performance?
* Increase the size of the embedding-vectors to e.g. 200. Does it affect performance?
* Try varying all the different hyper-parameters for the Recurrent Neural Network.
* Use Bayesian Optimization from Tutorial #19 to find the best choice of hyper-parameters.
* Use 'post' for padding and truncating in `pad_sequences()`. Does it affect the performance?
* Use individual characters instead of tokenized words as the vocabulary. You can then use one-hot encoded vectors for each character instead of using the embedding-layer.
* Use `model.fit_generator()` instead of `model.fit()` and make your own data-generator, which creates a batch of data using a random subset of `x_train_tokens`. The sequences must be padded so they all match the length of the longest sequence.
* Explain to a friend how the program works.

## License (MIT)

Copyright (c) 2018 by [Magnus Erik Hvass Pedersen](http://www.hvass-labs.org/)

Permission is hereby granted, free of charge, to any person obtaining a copy of this software and associated documentation files (the "Software"), to deal in the Software without restriction, including without limitation the rights to use, copy, modify, merge, publish, distribute, sublicense, and/or sell copies of the Software, and to permit persons to whom the Software is furnished to do so, subject to the following conditions:

The above copyright notice and this permission notice shall be included in all copies or substantial portions of the Software.

THE SOFTWARE IS PROVIDED "AS IS", WITHOUT WARRANTY OF ANY KIND, EXPRESS OR IMPLIED, INCLUDING BUT NOT LIMITED TO THE WARRANTIES OF MERCHANTABILITY, FITNESS FOR A PARTICULAR PURPOSE AND NONINFRINGEMENT. IN NO EVENT SHALL THE AUTHORS OR COPYRIGHT HOLDERS BE LIABLE FOR ANY CLAIM, DAMAGES OR OTHER LIABILITY, WHETHER IN AN ACTION OF CONTRACT, TORT OR OTHERWISE, ARISING FROM, OUT OF OR IN CONNECTION WITH THE SOFTWARE OR THE USE OR OTHER DEALINGS IN THE SOFTWARE.